Блокнот с соревнования на Kaggle

# Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,RandomForestRegressor
from sklearn.model_selection import KFold,ShuffleSplit,GridSearchCV

In [ ]:
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
import numpy as np
from geopy import distance
from sklearn.metrics import mean_absolute_percentage_error

# Загрузка данных

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/My Drive/train_geo.csv'

In [ ]:
test_path = '/content/drive/My Drive/test_geo.csv'

In [ ]:
train = pd.read_csv(train_path)

In [ ]:
test = pd.read_csv(test_path)

In [ ]:
train.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
test.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [ ]:
train.head()

,ID,vendorid,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,tariff,dropoff_longitude,dropoff_latitude,target,gc_distance,geo_distance,year,month,day,hour,minute,second
0,0,1,2016-05-24 22:39:23,1,-73.963608,40.774109,1,-73.920189,40.883194,34.84,12667.906522,12655.512912,2016,5,24,22,39,23
1,1,2,2016-05-01 05:19:10,1,-73.999313,40.733829,1,-74.013100,40.702332,13.56,3690.108427,3686.659617,2016,5,1,5,19,10
2,2,1,2016-05-21 16:45:08,1,-73.980980,40.741714,1,-73.920372,40.755959,20.30,5345.584256,5357.403877,2016,5,21,16,45,8
3,3,2,2016-05-30 18:48:46,1,-73.990868,40.714031,1,-73.975060,40.697948,12.96,2230.180084,2230.317980,2016,5,30,18,48,46
4,4,1,2016-05-19 17:30:51,1,-73.953140,40.775764,1,-73.983391,40.754620,29.75,3466.796550,3469.488549,2016,5,19,17,30,51


# Feature Engineering  

In [ ]:
def get_great_circle_distance(x):
  x1, y1, x2, y2 = x.pickup_latitude, x.pickup_longitude, x.dropoff_latitude, x.dropoff_longitude
  return distance.great_circle((x1,y1),(x2, y2)).m

In [ ]:
train['gc_distance'] = train.apply(lambda x: get_great_circle_distance(x), axis=1)
test['gc_distance'] = test.apply(lambda x: get_great_circle_distance(x), axis=1)

In [ ]:
def get_geodesic_distance(x):
  x1, y1, x2, y2 = x.pickup_latitude, x.pickup_longitude, x.dropoff_latitude, x.dropoff_longitude
  return distance.geodesic((x1,y1),(x2, y2)).m

In [ ]:
train['geo_distance'] = train.apply(lambda x: get_geodesic_distance(x), axis=1)
test['geo_distance'] = test.apply(lambda x: get_geodesic_distance(x), axis=1)

In [ ]:
# (train['gc_distance'] - train['geo_distance']).abs().mean()

5.143908302898595

In [ ]:
train

,ID,vendorid,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,tariff,dropoff_longitude,dropoff_latitude,target,gc_distance,geo_distance
0,0,1,2016-05-24T22:39:23.000,1,-73.963608,40.774109,1,-73.920189,40.883194,34.84,12667.906522,12655.512912
1,1,2,2016-05-01T05:19:10.000,1,-73.999313,40.733829,1,-74.013100,40.702332,13.56,3690.108427,3686.659617
2,2,1,2016-05-21T16:45:08.000,1,-73.980980,40.741714,1,-73.920372,40.755959,20.30,5345.584256,5357.403877
3,3,2,2016-05-30T18:48:46.000,1,-73.990868,40.714031,1,-73.975060,40.697948,12.96,2230.180084,2230.317980
4,4,1,2016-05-19T17:30:51.000,1,-73.953140,40.775764,1,-73.983391,40.754620,29.75,3466.796550,3469.488549
...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,4999995,1,2016-06-01T17:03:24.000,1,-74.001450,40.715759,1,-73.975601,40.745659,21.95,3974.592920,3973.980007
4999996,4999996,2,2016-06-14T13:19:59.000,2,-74.095520,40.850937,1,-73.981369,40.786488,16.56,11984.585911,11998.692099
4999997,4999997,2,2016-06-18T22:38:08.000,1,-74.002792,40.750179,1,-74.029800,40.729240,13.80,3255.552426,3257.423189
4999998,4999998,2,2016-06-04T04:24:14.000,6,-73.995758,40.767197,1,-73.982521,40.761890,7.30,1261.320683,1263.473657


In [ ]:
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'])
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'])

In [ ]:
def time(data):
  data['year'] = data['pickup_datetime'].dt.year
  data['month'] = data['pickup_datetime'].dt.month
  data['day'] = data['pickup_datetime'].dt.day
  data['hour'] = data['pickup_datetime'].dt.hour
  data['minute'] = data['pickup_datetime'].dt.minute
  data['second'] = data['pickup_datetime'].dt.second
  return data

In [ ]:
time(train)
time(test)

,ID,vendorid,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,tariff,dropoff_longitude,dropoff_latitude,gc_distance,geo_distance,year,month,day,hour,minute,second
0,5000000,2,2016-06-20 11:07:49,1,-73.994186,40.732819,1,-73.988060,40.738140,785.229907,785.509611,2016,6,20,11,7,49
1,5000001,1,2016-06-19 10:05:37,1,-73.979240,40.723610,1,-73.962349,40.702530,2742.478957,2741.722387,2016,6,19,10,5,37
2,5000002,1,2016-06-03 20:11:05,1,-73.988686,40.769032,1,-73.983795,40.775833,861.157785,860.791740,2016,6,3,20,11,5
3,5000003,2,2016-06-10 14:48:35,2,-73.981682,40.758560,1,-73.979889,40.749271,1043.849967,1042.564595,2016,6,10,14,48,35
4,5000004,2,2016-06-13 16:39:23,1,-73.972733,40.757290,1,-73.979088,40.755211,583.087959,584.221474,2016,6,13,16,39,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5999995,1,2016-06-13 18:47:02,1,-74.013893,40.713760,1,-74.003944,40.732872,2284.520827,2282.701980,2016,6,13,18,47,2
999996,5999996,1,2016-06-10 07:38:44,1,-73.957794,40.761421,1,-73.959663,40.773525,1355.083392,1353.378941,2016,6,10,7,38,44
999997,5999997,1,2016-06-30 00:08:44,1,-73.974762,40.735386,1,-73.993423,40.752537,2471.564677,2472.184778,2016,6,30,0,8,44
999998,5999998,1,2016-06-22 01:00:38,1,-73.815079,40.739269,1,-73.870163,40.770939,5824.839807,5831.480691,2016,6,22,1,0,38


In [ ]:
train.drop('pickup_datetime', axis = 1, inplace=True)
test.drop('pickup_datetime', axis = 1, inplace=True)

Тут я загружала данные, чтобы не новые переменные по несколько раз

In [ ]:
# train.to_csv('train_geo.csv', index=False)
# test.to_csv('test_geo.csv', index=False)

In [ ]:
X_train = train.drop('target', axis=1,)
y_train = train['target']

In [ ]:
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

Метрика SMAPE

In [ ]:
def calculate_smape(actual, predicted):

    # Convert actual and predicted to numpy
    # array data type if not already
    if not all([isinstance(actual, np.ndarray),
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)

    return  np.mean(
            np.abs(predicted - actual) /
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100


# XGBRegressor

In [ ]:
from xgboost import XGBRegressor

In [ ]:
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
my_model = XGBRegressor()
my_model.fit(Xtrain, ytrain)

train_pred = my_model.predict(Xtrain)
val_pred = my_model.predict(Xval)

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [ ]:
def objective(space):
    clf = XGBRegressor(
        n_estimators=int(space['n_estimators']), # Важно преобразовать, так как hp.quniform возвращает float
        max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        reg_alpha=space['reg_alpha'], # Убрал int, чтобы не обрезать
        reg_lambda=space['reg_lambda'],
        colsample_bytree=space['colsample_bytree'], # Убрал int
        min_child_weight=int(space['min_child_weight']),
        random_state=space['seed'], # Для воспроизводимости
        n_jobs=-1 # Использовать все доступные ядра
    )
    evaluation = [( Xtrain, ytrain), (Xval, yval)]

    clf.fit(Xtrain, ytrain,
            eval_set=evaluation,
            verbose=False)

    y_pred = clf.predict(Xval)

    smape = calculate_smape(np.array(yval), np.array(y_pred))
    return smape

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

100%|██████████| 100/100 [7:04:17<00:00, 254.58s/trial, best loss: 14.0265673131006]


In [ ]:
best_hyperparams

{'colsample_bytree': 0.9947648216052315,
 'gamma': 1.2274684207463673,
 'max_depth': 17.0,
 'min_child_weight': 1.0,
 'reg_alpha': 145.0,
 'reg_lambda': 0.08712342039127541}

Посчитанные параметры

{'colsample_bytree': 0.9947648216052315,
 'gamma': 1.2274684207463673,
 'max_depth': 17.0,
 'min_child_weight': 1.0,
 'reg_alpha': 145.0,
 'reg_lambda': 0.08712342039127541}

In [ ]:
my_model = XGBRegressor(colsample_bytree = 0.9947648216052315, gamma = 1.2274684207463673, max_depth= 17, min_child_weight= 1, reg_alpha = 145, reg_lambda= 0.08712342039127541)
my_model.fit(Xtrain, ytrain)

train_pred = my_model.predict(Xtrain)
val_pred = my_model.predict(Xval)

In [ ]:
print('Train: ', calculate_smape(np.array(ytrain), np.array(train_pred)))
print('Val: ',calculate_smape(np.array(yval), np.array(val_pred)))

Train:  12.766752490392088
Val:  14.047402350973162


In [ ]:
test_pred = my_model.predict(test)